In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
import warnings; warnings.simplefilter('ignore', FutureWarning)
import tensorflow as tf
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

import pymongo
from pymongo import MongoClient
from avian_vocalizations.evaluator import ParamSpace, EvaluatorFactory
from hyperopt.mongoexp import MongoTrials, MongoCtrl, MongoJobs, MongoWorker
from hyperopt import hp, fmin, tpe
from vastai.api import VastClient
from toolbench.tools import print_module_versions
print_module_versions(globals())

Using TensorFlow backend.


             IPython  {'7.8.0'}
          tensorflow  {'1.14.0'}
             pymongo  {'3.9.0'}
 avian_vocalizations  {'0.0.1'}
            hyperopt  {'0.2.1'}
           toolbench  {'0.0.1'}


In [3]:
vast = VastClient().authenticate()

api_key_file:  ~/.vast_api_key
Initializing vast.ai client with api_key from /home/sam/.vast_api_key.
Already logged in.


## Test tunnel to mongodb

In [4]:
instance = vast.get_instance(400808)
remote = instance.pb_remote

In [5]:
print(instance.ssh_connection_command)

ssh root@ssh4.vast.ai -p 10808 -i /home/sam/.ssh/vastai


In [6]:
print(remote['tail']('/var/log/mongodb/mongodb.log','-n2'))

2019-10-26T21:02:47.435+0000 I NETWORK  [initandlisten] waiting for connections on port 27017
2019-10-26T21:02:48.017+0000 I FTDC     [ftdc] Unclean full-time diagnostic data capture shutdown detected, found interim file, some metrics may have been lost. OK



In [7]:
with instance.get_tunnel(27017,27017) as tunn:
    mongo = MongoClient('localhost', 27017)
    print(mongo.list_database_names())

['admin', 'avian_vocalizations', 'config', 'hyperopt', 'local']


In [10]:
instance._tunnels

{27017: <SshTunnel (defunct)>}

In [11]:
instance._ssh_machine_alive

True

## Hot update evaluator code

In [77]:
remote.sftp.put('avian_vocalizations/evaluator.py',
                '/root/avian_vocalizations/avian_vocalizations/evaluator.py')

<SFTPAttributes: [ size=4895 uid=0 gid=0 mode=0o100644 atime=1572122598 mtime=1572123513 ]>

In [68]:
remote.sftp.put('avian_vocalizations/data.py',
                '/root/avian_vocalizations/avian_vocalizations/data.py')

<SFTPAttributes: [ size=13787 uid=0 gid=0 mode=0o100644 atime=1572122143 mtime=1572122578 ]>

In [ ]:
remote.sftp.put('avian_vocalizations/model.py',
                '/root/avian_vocalizations/avian_vocalizations/model.py')

## Update code repo

In [9]:
with remote.cwd('/root/avian_vocalizations'):
    print(remote['git']('pull'))

Already up to date.



## Run some trials

In [12]:
tunnel = instance.get_tunnel(27017, 27017)

In [65]:
trials = MongoTrials('mongo://localhost:27017/avian_vocalizations/jobs', exp_key='test_experiment')

In [ ]:
hp_space = ParamSpace(n_frames=128, dropout_rate=hp.uniform('dropout',0,.5),batch_size=64)

evaluator = EvaluatorFactory(n_splits=3, n_epochs=10, data_dir='data')
best = fmin(evaluator, hp_space, trials=trials, algo=tpe.suggest, max_evals=2 )

over-writing old domain trials attachment


100%|██████████| 1/1 [00:03<00:00,  3.48s/it, best loss: 4.312834680080414]


In [64]:
# trials.delete_all()

## Cleanup

In [ ]:
tunnel.close()